In [1]:
import os

import pandas as pd

In [4]:
PROJECT_FOLDER = os.path.abspath('..')
DATA_FOLDER = os.path.join(PROJECT_FOLDER, 'local', 'datasets_biblioteki', 'datasets_2')

In [9]:
readers = pd.read_csv(f'{DATA_FOLDER}/readers.csv', sep=';', encoding='cp1251', header=None)
readers.head()

/home/magleb/rec_sys_book/local/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,0,1,2,3,4
0,366670,14.10.1974,"Обручевский, 10",NaN,NaN
1,175006,20.03.1978,"г. Москва, ул. Саранская, д. 7,",NaN,NaN
2,334805,03.12.1985,"г. Москва, ул. Ясеневая, д. 12, корп. 1,",NaN,NaN
3,364422,15.03.1992,"г. Владивосток, ул. Анны Щетининой, д. 20",NaN,NaN
4,37574,18.11.1981,"г. Москва, ул. Менжинского, д. 28, корп. 2,",NaN,NaN


In [14]:
user_book_interactions = pd.read_csv(f'{PROJECT_FOLDER}/local/raw_data/user_book_interaction_full.csv')
user_book_interactions.head()

,dt,user_id,book_id
0,2021-06-01,1,1163786
1,2021-06-01,1,66612
2,2021-06-01,1,1034596
3,2021-06-01,1,1152760
4,2021-06-01,1,16168


In [15]:
books_info = pd.read_csv(f'{PROJECT_FOLDER}/data/books.csv')
books_info.head()

,id,bbk,year,author,annotation,volume,title_additional,age_restriction,rubrics,author_id,title,available
0,1272541,84(2Рос=Рус)1,2013,Федор Достоевский,"""Бесы""- одно из наиболее трагических, загадочн...",604,NaN,NaN,Художественная литература,45567.0,Бесы,2
1,1960078,84(4Вел),2017,Роберт Гэлбрейт,Робин Эллакотт получает с курьером таинственны...,540,NaN,16.0,Художественная литература,41515.0,На службе зла,1
2,26991,84(7Сое),2004,Фэнни Флэгг,Роман американской писательницы Ф.Флэгг расска...,428,NaN,NaN,Художественная литература,116027.0,"Жареные зеленые помидоры в кафе ""Полустанок""",7
3,1781433,84(7Сое),2020,Ричард Руссо,"Майлз Роби двадцать лет готовит бургеры в ""Имп...",606,NaN,16.0,Художественная литература,141830.0,Эмпайр Фоллз,11
4,1687060,84(7США),2019,Р. Руссо,NaN,608,NaN,NaN,Художественная литература,141830.0,Эмпайр Фоллз,1


In [16]:
cat_1 = pd.read_csv(f'{DATA_FOLDER}/cat_1.csv', sep=';', encoding='cp1251')
cat_2 = pd.read_csv(f'{DATA_FOLDER}/cat_2.csv', sep=';', encoding='cp1251')

In [99]:
cleared_columns = ['readerID', 'catalogueRecordID', 'startDate']
books_circulations = set()
user_circulations = set()
cleared_circulations = pd.DataFrame(columns=cleared_columns)

for i in range(1, 17):
    circulation = pd.read_csv(f'{DATA_FOLDER}/circulaton_{i}.csv', sep=';', encoding='cp1251')
    circulation = circulation[circulation['state'] == 'На руках']
    cleared_circulations = cleared_circulations.append(circulation[circulation['state'] == 'На руках'][cleared_columns])
    books_circulations.update(circulation['catalogueRecordID'])
    user_circulations.update(circulation['readerID'])

cleared_circulations.columns = ['user_id', 'book_id', 'dt']
cleared_circulations.head()

,user_id,book_id,dt
756,118736,5186,01.09.2019
757,118736,316682,01.09.2019
1766,180893,751608,01.09.2019
2070,201318,585605,01.09.2019
2071,201318,41656,01.09.2019


In [ ]:
cleared_circulations.to_csv(f'{PROJECT_FOLDER}/data/circulations.csv', index=False)

In [6]:
cleared_circulations = pd.read_csv(f'{PROJECT_FOLDER}/data/train_interactions.csv')

In [17]:
# READERS
print(len(set(readers[0])), 'READERS')
print(len(set(readers[0]) - set(user_book_interactions['user_id'])), 'READERS - INTERACTIONS')
print(len(set(readers[0]) - user_circulations), 'READERS - CIRCULATIONS')
print('\n')
print(len(user_circulations), 'CIRCULATIONS')
print(len(user_circulations - set(user_book_interactions['user_id'])), 'CIRCULATIONS - INTERACTIONS')


569822 READERS
569822 READERS - INTERACTIONS


NameError: name 'user_circulations' is not defined

In [23]:
# BOOKS
print(len(set(user_book_interactions['book_id'])), 'INTERACTIONS')
print('\n')
print(len(set(books_info['id'])), 'SMALL JSON')
print('\n')
print(len(set(cleared_circulations['book_id'])), 'CIRCULATIONS')
print(len(set(cleared_circulations['book_id']) - set(books_info['id'])), 'CIRCULATIONS - SMALL JSON')
print(len(set(cleared_circulations['book_id']) - set(user_book_interactions['book_id'])), 'CIRCULATIONS - INTERACTIONS')

3943 INTERACTIONS


4682 SMALL JSON


193069 CIRCULATIONS
188848 CIRCULATIONS - SMALL JSON
189278 CIRCULATIONS - INTERACTIONS


In [181]:
test_ids = user_book_interactions['user_id'].value_counts().index.tolist()[:-8]
user_book_interactions_train = pd.DataFrame(columns=user_book_interactions.columns)
user_book_interactions_test = pd.DataFrame(columns=['user_id', 'book_id_1', 'book_id_2', 'book_id_3', 'book_id_4', 'book_id_5'])

for user_id in test_ids:
    inter_train_data = user_book_interactions[user_book_interactions['user_id'] == user_id].reset_index(drop=True)
    inter_test_data = inter_train_data[-5:].reset_index(drop=True)
    test_data = pd.DataFrame({'user_id': inter_test_data['user_id'][0],
                              **{f'book_id_{i + 1}': book_id for (i, book_id) in enumerate(inter_test_data['book_id'])}}, index=[0])
    train_data = inter_train_data.drop(index=list(range(len(inter_train_data) - 1, len(inter_train_data) - 6, -1)))
    user_book_interactions_train = user_book_interactions_train.append(train_data)
    user_book_interactions_test = user_book_interactions_test.append(test_data)

user_book_interactions_train.reset_index(drop=True)
user_book_interactions_test.reset_index(drop=True)

,user_id,book_id_1,book_id_2,book_id_3,book_id_4,book_id_5
0,34,1866509,1882247,1812326,2523335,1205629
1,23,80548,967568,1897994,2011853,734767
2,51,318612,16371,1201809,313152,349614
3,64,1824182,384273,1609359,836758,1824122
4,30,331155,6532,1791079,2002568,1818924
...,...,...,...,...,...,...
84,68,814603,828096,118466,1624999,2088574
85,28,139354,453682,9410,9481,13954
86,73,1800154,1961841,322503,841969,218331
87,88,374514,173502,346259,173584,458511


In [185]:
user_book_interactions_train.to_csv(f'{PROJECT_FOLDER}/data/user_book_interaction_train.csv', index=False)
user_book_interactions_test.to_csv(f'{PROJECT_FOLDER}/data/test.csv', index=False)